## LangChain

In [ ]:
#openai

from langchain.llms import OpenAI

import os
from dotenv import load_dotenv

load_dotenv()

llm = OpenAI(model_name="gpt-3.5-turbo",openai_api_key=os.getenv("OPENAI_API_KEY"))




In [ ]:
# Gemini
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=os.getenv("GOOGLE_API_KEY"))




In [ ]:
prompt=" Conte uma historia sobre apredizado de máquina"
llm.invoke(prompt) #chama minha ia passando o promp

In [ ]:
#gerando o texto parte a parte

prompt=" Conte uma historia sobre apredizado de máquina"
for trecho in llm.stream(prompt):
    llm.invoke(trecho, end="") #chama minha ia passando o promp


In [ ]:
#fazer perguntas em lote
perguntas=[
    "o que é Brasil?",
    "o que é brasileiros?",
    "o que é samba?"
]

llm.abatch(perguntas)

## ChatModels


In [ ]:
from langchain_openai import ChatOpenAI #chatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage #modelos de cofigura a messangem

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [ ]:
mensagens=[
    #SystemMessage define o comportamento do assistente
    SystemMessage(content="Você é um asssistente que responde com irônia e zueira"),
    HumanMessage(content="Qual o papel da memória cache?")
]

resposta= chat.invoke(mensagens)

resposta #tente ver o resposta.contate e resposta.response_metadata

## Prompt few shot

Few-shot prompting é uma técnica usada com modelos de linguagem (como o ChatGPT) onde você dá alguns exemplos de como o modelo deve responder antes de fazer sua pergunta real.

A ideia é ensinar o modelo pelo exemplo, sem precisar reprogramar nada.

Tipo de Prompt	Exemplo	Característica
Zero-shot	"Traduza para inglês: Olá mundo"	Nenhum exemplo
One-shot	+1 exemplo antes da tarefa	Um exemplo
Few-shot	+2 ou mais exemplos	Alguns exemplos
Fine-tuning	Treinamento com muitos exemplos	Requer ajuste do modelo


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

chat=ChatOpenAI(model="gpt.3.5-turbo")

In [ ]:
mensagens=[
    SystemMessage(content="voce sera ironico"),
    HumanMessage(content="o que é tv?"),
    AIMessage(content="televisão é um dispositivo que assitimos")
]

In [ ]:
#debug
import langchain

langchain.debug=True

chat.invoke(mensagens)

## Cacheamento

 O cacheamento em LangChain é um recurso que permite evitar chamadas repetidas à API (como OpenAI ou Gemini) quando o mesmo prompt já foi processado antes. Isso ajuda a:

✅ Economizar tokens e custos
✅ Aumentar a velocidade de resposta
✅ Evitar chamadas desnecessárias à API
🔹 Como funciona o cache no LangChain?
LangChain pode armazenar os resultados de chamadas LLM em cache usando diferentes backends, como:

Em memória (padrão, útil para testes rápidos)
SQLite (persistente entre execuções)
Redis (para ambientes distribuídos)

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
#cacheamneto
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

chat= ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
mensagens=[
    SystemMessage(content="Você é um assistente irritante"),
    HumanMessage(content="Qual o primeiro dia da semana?")
]

### Cacheamento em InmemoryCache 
 um tipo de cache volátil (não persistente) que guarda os resultados das chamadas LLM na memória RAM enquanto o programa está rodando. Se você rodar o mesmo prompt duas vezes, ele retorna a resposta da primeira vez, sem chamar a API de novo.

 ´´´´ from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

In [ ]:
#Cacheamento em InmemoryCache 
# from langchain.cache import InMemoryCache
#from langchain.globals import set_llm_cache 

import time


set_llm_cache(InMemoryCache())


start = time.time()


chat.invoke(mensagens)
print(time)

### Cache em banco de dados
posso te mostrar como usar cache persistente com SQLite ou até medir o tempo de resposta com e sem cache

In [ ]:
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from dotenv import load_dotenv
import os
import time

# Carrega variáveis do .env
load_dotenv()

# Ativa o cache persistente com SQLite
set_llm_cache(SQLiteCache(database_path=".langchain_cache.db"))

# Inicializa o modelo
chat = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"))

# Define as mensagens
mensagens = [
    SystemMessage(content="Você é um assistente que responde com ironia e zueira"),
    HumanMessage(content="Qual o papel da memória cache?")
]

# Primeira chamada (vai consultar a API e salvar no cache)
start = time.time()
resposta1 = chat.invoke(mensagens)
print("Resposta 1:", resposta1.content)
print("Tempo 1:", time.time() - start, "segundos")

# Segunda chamada (vai usar o cache do SQLite)
start = time.time()
resposta2 = chat.invoke(mensagens)
print("Resposta 2:", resposta2.content)
print("Tempo 2:", time.time() - start, "segundos")
